In [22]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

import os
from distutils.dir_util import copy_tree, remove_tree

from PIL import Image
from random import randint

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef as MCC
from sklearn.metrics import balanced_accuracy_score as BAS
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow_addons as tfa
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG
from tensorflow.keras.layers import SeparableConv2D, BatchNormalization, GlobalAveragePooling2D


print("TensorFlow Version:", tf.__version__)

TensorFlow Version: 2.13.0


In [33]:
WORK_DIR = './Alzheimers-OASIS'

CLASSES = [ 'Mild Dementia', 'Moderate Dementia', 'Non Demented', 'Very mild Dementia',]

IMG_SIZE = 248
IMAGE_SIZE = [248, 248]
DIM = (IMG_SIZE, IMG_SIZE)

In [36]:
ZOOM = [.99, 1.01]
BRIGHT_RANGE = [0.8, 1.2]
HORZ_FLIP = True
FILL_MODE = "constant"
DATA_FORMAT = "channels_last"

work_dr = IDG(rescale = 1./255, brightness_range=BRIGHT_RANGE, zoom_range=ZOOM, data_format=DATA_FORMAT, fill_mode=FILL_MODE, horizontal_flip=HORZ_FLIP)

train_data_gen = work_dr.flow_from_directory(directory=WORK_DIR, target_size=DIM, batch_size=9000, shuffle=False)

Found 86437 images belonging to 4 classes.


In [37]:
train_data, train_labels = train_data_gen.next()

MemoryError: Unable to allocate 6.19 GiB for an array with shape (9000, 248, 248, 3) and data type float32

In [31]:
print(train_data.shape, train_labels.shape)

(6000, 173, 173, 3) (6000, 4)


In [32]:
train_data, test_data, train_labels, test_labels = train_test_split(train_data, train_labels, test_size = 0.4, random_state=42)
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size = 0.4, random_state=42)

MemoryError: Unable to allocate 1.20 GiB for an array with shape (3600, 173, 173, 3) and data type float32

In [14]:
vgg19_model = VGG19(input_shape=(173, 173, 3), include_top=False, weights="imagenet")

In [15]:
for layer in vgg19_model.layers:
    layer.trainable=False

In [ ]:
custom_vgg19_model = Sequential([
        vgg19_model,
        Dropout(0.5),
        GlobalAveragePooling2D(),
        Flatten(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        BatchNormalization(),
        Dense(4, activation='softmax')        
    ], name = "vgg19model")

In [17]:
import keras.backend as K
def f1_score(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [18]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),  
      tf.keras.metrics.AUC(name='auc'),
        f1_score,
]

# CALLBACKS = rop_callback
    
custom_vgg19_model.compile(optimizer='rmsprop',
                              loss=tf.losses.CategoricalCrossentropy(),
                              metrics=METRICS)

custom_vgg19_model.summary()

Model: "vgg19model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 5, 5, 512)         20024384  
                                                                 
 dropout (Dropout)           (None, 5, 5, 512)         0         
                                                                 
 global_average_pooling2d (  (None, 512)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 batch_normalization (Batch  (None, 512)               2048      
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 512)               2

In [19]:
EPOCHS = 3

history = custom_vgg19_model.fit(train_data, train_labels, validation_data=(val_data, val_labels), epochs=EPOCHS,steps_per_epoch=128)

Epoch 1/3
128/128 [==============================] - 575s 5s/step - loss: 1.4427 - accuracy: 0.7362 - precision: 0.4531 - recall: 0.2664 - auc: 0.6598 - f1_score: 0.3320 - val_loss: 0.9414 - val_accuracy: 0.7530 - val_precision: 0.8250 - val_recall: 0.0153 - val_auc: 0.8975 - val_f1_score: 0.0289
Epoch 2/3
128/128 [==============================] - 575s 5s/step - loss: 0.9621 - accuracy: 0.8107 - precision: 0.6602 - recall: 0.5006 - auc: 0.8402 - f1_score: 0.5675 - val_loss: 0.7754 - val_accuracy: 0.8523 - val_precision: 0.7492 - val_recall: 0.6153 - val_auc: 0.9129 - val_f1_score: 0.6747
Epoch 3/3
128/128 [==============================] - 556s 4s/step - loss: 0.8389 - accuracy: 0.8274 - precision: 0.6761 - recall: 0.5944 - auc: 0.8770 - f1_score: 0.6325 - val_loss: 0.6746 - val_accuracy: 0.8691 - val_precision: 0.7592 - val_recall: 0.6977 - val_auc: 0.9303 - val_f1_score: 0.7263


In [20]:
#Evaluating the model on the data

#train_scores = model.evaluate(train_data, train_labels)
#val_scores = model.evaluate(val_data, val_labels)
test_scores = custom_vgg19_model.evaluate(test_data, test_labels)

#print("Training Accuracy: %.2f%%"%(train_scores[1] * 100))
#print("Validation Accuracy: %.2f%%"%(val_scores[1] * 100))
print("Testing Accuracy: %.2f%%"%(test_scores[1] * 100))

113/113 [==============================] - 382s 3s/step - loss: 0.6891 - accuracy: 0.8631 - precision: 0.7473 - recall: 0.6833 - auc: 0.9236 - f1_score: 0.7137
Testing Accuracy: 86.31%


In [2]:
pred_labels = custom_vgg19_model.predict(test_data)

NameError: name 'custom_vgg19_model' is not defined

In [ ]:
#Print the classification report of the tested data

#Since the labels are softmax arrays, we need to roundoff to have it in the form of 0s and 1s,
#similar to the test_labels
def roundoff(arr):
    """To round off according to the argmax of each predicted label array. """
    arr[np.argwhere(arr != arr.max())] = 0
    arr[np.argwhere(arr == arr.max())] = 1
    return arr

for labels in pred_labels:
    labels = roundoff(labels)

print(classification_report(test_labels, pred_labels, target_names=CLASSES))